# Web Scraping

## Imports

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import io
from PIL import Image
import os

In [2]:
french_paintings = pd.read_csv('french_paintings_metadata.csv')
french_paintings

,Unnamed: 0,index,AUTHOR,BORN-DIED,TITLE,DATE,TECHNIQUE,LOCATION,URL,FORM,TYPE,SCHOOL,TIMEFRAME
0,0,493,"ALLEGRAIN, Etienne","(b. 1644, Paris, d. 1736, Paris)",Landscape with the Finding of Moses,-,"Oil on canvas, 88 x 115 cm","The Hermitage, St. Petersburg",https://www.wga.hu/html/a/allegrai/etienne/lan...,painting,landscape,French,1651-1700
1,1,494,"ALLEGRAIN, Etienne","(b. 1644, Paris, d. 1736, Paris)",River Landscape,-,Oil on canvas : 118 x 92 cm,"Musée National du Château, Versailles",https://www.wga.hu/html/a/allegrai/etienne/riv...,painting,landscape,French,1651-1700
2,2,495,"ALLEGRAIN, Etienne","(b. 1644, Paris, d. 1736, Paris)",River Landscape,-,Oil on canvas : 86 x 136 cm,"Musée du Louvre, Paris",https://www.wga.hu/html/a/allegrai/etienne/riv...,painting,landscape,French,1651-1700
3,3,1304,"ANGRAND, Charles","(b. 1854, Criquetot-sur-Ouville, d. 1926, Rouen)",Couple in the Street,1887,"Oil on canvas on card, 39 x 33 cm","Musée d'Orsay, Paris",https://www.wga.hu/html/a/angrand/couple.html,painting,genre,French,1851-1900
4,4,1305,"ANGRAND, Charles","(b. 1854, Criquetot-sur-Ouville, d. 1926, Rouen)",The Western Railway Leaving Paris,1886,"Oil on canvas, 75 x 92 cm",Private collection,https://www.wga.hu/html/a/angrand/railway.html,painting,landscape,French,1851-1900
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4321,4321,51283,"ZIEM, Félix","(b. 1821, Beaune, d. 1911, Paris)","Riva degli Schiavone, Venice",c. 1847,"Oil on canvas, 68 x 105 cm",Private collection,https://www.wga.hu/html/z/ziem/ziem6.html,painting,landscape,French,1851-1900
4322,4322,51284,"ZIEM, Félix","(b. 1821, Beaune, d. 1911, Paris)","San Simeone Piccolo, Venice",1865,"Oil on canvas, 54 x 69 cm",Private collection,https://www.wga.hu/html/z/ziem/ziem7.html,painting,landscape,French,1851-1900
4323,4323,51285,"ZIEM, Félix","(b. 1821, Beaune, d. 1911, Paris)","The Rialto Bridge, Venice",-,"Oil on panel, 72 x 93 cm",Private collection,https://www.wga.hu/html/z/ziem/ziem8.html,painting,landscape,French,1851-1900
4324,4324,51286,"ZIEM, Félix","(b. 1821, Beaune, d. 1911, Paris)",A View of Venice,-,"Oil on panel, 60 x 81 cm",Private collection,https://www.wga.hu/html/z/ziem/ziem9.html,painting,landscape,French,1851-1900


In [3]:
for index, row in french_paintings.iterrows():
    # download page
    page = row['URL']
    
    result = requests.get(page)
    
    # if successful parse the download into a BeautifulSoup object, which allows easy manipulation 
    if result.status_code == 200:
        soup = BeautifulSoup(result.content, "html.parser")
        
    # find the object which has the attribute cellpadding == 5
    table = soup.find("table", {"cellpadding" : 5})
    # loop through all the rows
    if table is not None:
        for link in table.find_all('a', href=True):
            french_paintings.at[index, 'URL'] = 'images/'+os.path.basename(link["href"])
            if link['href'].endswith('.jpg') or link['href'].endswith('.png') or link['href'].endswith('.jpeg'):
               
                img_data = requests.get('https://www.wga.hu/' + link['href']).content
                
                image_bytes = io.BytesIO(img_data)
                im = Image.open(image_bytes) 
                im = im.save('../images/'+os.path.basename(link['href']))
french_paintings.to_csv("french_paintings.csv")

KeyboardInterrupt: 